In [1]:
import scipy.sparse as sp
import numpy as np
import torch 
import tensorflow as tf

# Load the sparse matrix from the .npz file
A_sparse_scipy = sp.load_npz('A_sparse_scipy.npz')
p_dense_numpy = np.load('A_numpy.npy')

2024-11-04 20:30:28.325725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730752228.341338  260487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730752228.346093  260487 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 20:30:28.361766: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
A_sparse_scipy = A_sparse_scipy.tocoo()

# Extract row, column, and data arrays from the SciPy sparse matrix
row = torch.from_numpy(A_sparse_scipy.row).long()
col = torch.from_numpy(A_sparse_scipy.col).long()
values = torch.from_numpy(A_sparse_scipy.data)

# Stack row and column indices to create a 2 x N tensor of indices
indices = torch.stack([row, col])

# Create the PyTorch sparse tensor
A_torch = torch.sparse_coo_tensor(indices, values, size=A_sparse_scipy.shape, dtype=torch.float32, device="cuda:0")
A_torch = A_torch.to_sparse_csr()
p_torch = torch.tensor(p_dense_numpy, dtype=torch.float32, device="cuda:0")

In [8]:
%timeit torch.sparse.mm(A_torch, p_torch)

67.2 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
# Extract row indices, column indices, and data values
indices = np.vstack((A_sparse_scipy.row, A_sparse_scipy.col)).T
values = A_sparse_scipy.data
dense_shape = A_sparse_scipy.shape

# Create a TensorFlow SparseTensor on GPU
with tf.device('/GPU:0'):  # Specify the GPU device
    A_tf = tf.SparseTensor(indices=tf.convert_to_tensor(indices, dtype=tf.int64),
                                   values=tf.convert_to_tensor(values, dtype=tf.float32),
                                   dense_shape=tf.convert_to_tensor(dense_shape, dtype=tf.int64))
    p_tf = tf.convert_to_tensor(p_dense_numpy, dtype=tf.float32)

In [10]:
%timeit tf.sparse.sparse_dense_matmul(A_tf, tf.reshape(p_tf, (-1, 1)))

275 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
